In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from thalamus_merfish_analysis import ccf_plots as cplot
from thalamus_merfish_analysis import ccf_images as cimg
from thalamus_merfish_analysis import abc_load as abc

get_ipython().run_line_magic('matplotlib', 'inline') 

## Load brain3 data

In [2]:
obs = abc.get_combined_metadata(realigned=True)
obs = abc.label_thalamus_spatial_subset(obs, realigned=True, filter_cells=True)

ccf_label = 'parcellation_substructure_realigned'
nn_classes = [
    "31 OPC-Oligo",
    "30 Astro-Epen",
    "33 Vascular",
    "34 Immune",
]
# subset just the neurons
obs_neurons = obs[~obs['class'].isin(nn_classes)]

In [5]:
th_names = abc.get_thalamus_substructure_names()
th_subregion_names = list(set(th_names).difference(['TH-unassigned']))


In [81]:
# define celltype lists based on strict spatial subset
# obs_th_neurons = obs_neurons[obs_neurons[ccf_label].isin(th_names)]
obs_th_neurons = obs_neurons[obs_neurons['parcellation_substructure_realigned'].isin(th_names) |
                             obs_neurons['parcellation_substructure'].isin(th_names)]
th_celltypes = dict()
th_celltypes['subclass'] = obs_th_neurons['subclass'].value_counts().loc[lambda x: x>100].index
print(f"{len(th_celltypes['subclass'])=}")

th_celltypes['supertype'] = obs_th_neurons['supertype'].value_counts().loc[lambda x: x>20].index
print(f"{len(th_celltypes['supertype'])=}")

th_celltypes['cluster'] = obs_th_neurons['cluster'].value_counts().loc[lambda x: x>10].index
print(f"{len(th_celltypes['cluster'])=}")

len(th_celltypes['subclass'])=29
len(th_celltypes['supertype'])=128
len(th_celltypes['cluster'])=448


## matching metrics

In [44]:
from sklearn.metrics import f1_score, precision_score, recall_score

def get_nucleus_celltype_match(obs, ccf_label, ccf_name, celltype_label, celltype_name):
    nucleus = obs[ccf_label] == ccf_name
    celltype = obs[celltype_label] == celltype_name
    record = {
        'nucleus_precision': precision_score(nucleus, celltype),
        'nucleus_coverage': recall_score(nucleus, celltype),
        'nucleus_f1': f1_score(nucleus, celltype),
    }
    return record
    
def get_nucleus_celltype_metrics(obs, ccf_label, celltype_label, 
                                 ccf_list=None, celltype_list=None):
    if celltype_list is None:
        celltype_list = obs[celltype_label].unique()
    # else: obs = obs[obs[celltype_label].isin(celltype_list)]
    if ccf_list is None:
        ccf_list = obs[ccf_label].unique()
    # else: obs = obs[obs[ccf_label].isin(ccf_list)]
    # could subset like this, but need to track the negatives...
    
    records = []
    for celltype_name in celltype_list:
        celltype = obs[celltype_label] == celltype_name
        for ccf_name in ccf_list:
            nucleus = obs[ccf_label] == ccf_name
            tp = (nucleus & celltype).sum()
            fp = (~nucleus & celltype).sum()
            fn = (nucleus & ~celltype).sum()
            recall = tp/(tp+fn)
            precision = tp/(tp+fp)
            jaccard = tp/(tp+fp+fn)
            f1 = 2*recall*precision/(recall+precision)
            if precision>0.5 or recall>0.5 or f1>0.4:
                record = {
                    'nucleus': ccf_name,
                    'celltype': celltype_name,
                    'nucleus_precision': precision,
                    'nucleus_recall': recall,
                    'nucleus_f1': f1,
                    'jaccard':jaccard
                }
                records.append(record)
    return pd.DataFrame.from_records(records)

In [67]:
subclass_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'subclass', 
                                           ccf_list=th_subregion_names,
                                           celltype_list=th_celltypes['subclass'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [68]:
subclass_set = subclass_df.query('nucleus_precision>0.6 & nucleus_recall>0.15')
subclass_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
24,MH,145 MH Tac2 Glut,0.801533,0.979953,0.881809
38,AV,148 AV Col27a1 Glut,0.867015,0.814956,0.840180
23,RT,093 RT-ZI Gnb3 Gaba,0.641743,0.928089,0.758801
41,AD,147 AD Serpinb7 Glut,0.660504,0.826498,0.734236
42,LH,146 LH Pou4f1 Sox1 Glut,0.718574,0.593338,0.649979


In [61]:
subclass_region_names = set(subclass_set['nucleus']).difference(['LGd-sh'])
nonsubclass_region_names = set(th_subregion_names).difference(subclass_region_names)

In [69]:
supertype_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'supertype', 
                                           ccf_list=nonsubclass_region_names,
                                           celltype_list=th_celltypes['supertype'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [70]:
supertype_set = supertype_df.query('nucleus_precision>0.6 & nucleus_recall>0.15')
supertype_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
8,PF,0675 PF Fzd5 Glut_2,0.636688,0.637826,0.637257
11,RE,0670 RE-Xi Nox4 Glut_3,0.757943,0.543722,0.633205
9,LD,0658 TH Prkcd Grin2c Glut_5,0.639803,0.534702,0.582550
7,MD,0663 TH Prkcd Grin2c Glut_10,0.608185,0.379051,0.467027
22,PT,0648 PVT-PT Ntrk1 Glut_6,0.727575,0.162222,0.265294


In [71]:
cluster_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'cluster', 
                                           ccf_list=nonsubclass_region_names,
                                           celltype_list=th_celltypes['cluster'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [72]:
cluster_set = cluster_df.query('nucleus_precision>0.6 & nucleus_recall>0.1')
cluster_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
2,VPM,2649 TH Prkcd Grin2c Glut_1,0.658752,0.660979,0.659863
17,AMd,2674 TH Prkcd Grin2c Glut_9,0.624071,0.487805,0.547588
6,LD,2658 TH Prkcd Grin2c Glut_5,0.750910,0.423409,0.541492
16,PT,2677 TH Prkcd Grin2c Glut_10,0.739255,0.382222,0.503906
3,MD,2678 TH Prkcd Grin2c Glut_10,0.769601,0.367343,0.497311
7,VM,2687 TH Prkcd Grin2c Glut_13,0.665842,0.360429,0.467691
9,RE,2698 RE-Xi Nox4 Glut_3,0.813364,0.292582,0.430357
5,PO,2663 TH Prkcd Grin2c Glut_6,0.746333,0.288176,0.415801
11,PF,2723 PF Fzd5 Glut_2,0.703325,0.196639,0.307348
14,PF,2724 PF Fzd5 Glut_2,0.708556,0.189489,0.299013


## original alignment

In [82]:
ccf_label = 'parcellation_substructure'

In [83]:
subclass_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'subclass', 
                                           ccf_list=th_subregion_names,
                                           celltype_list=th_celltypes['subclass'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [84]:
subclass_set = subclass_df.query('nucleus_precision>0.6 & nucleus_recall>0.15')
subclass_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
27,MH,145 MH Tac2 Glut,0.937819,0.976713,0.956871
26,RT,093 RT-ZI Gnb3 Gaba,0.836096,0.979011,0.901927
41,AD,147 AD Serpinb7 Glut,0.870588,0.920071,0.894646
42,LH,146 LH Pou4f1 Sox1 Glut,0.798311,0.751767,0.774340
40,AV,148 AV Col27a1 Glut,0.703833,0.795798,0.746995
29,ZI,101 ZI Pax6 Gaba,0.622503,0.540986,0.578889


In [85]:
subclass_region_names = set(subclass_set['nucleus']).difference(['LGd-sh'])
nonsubclass_region_names = set(th_subregion_names).difference(subclass_region_names)

In [86]:
supertype_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'supertype', 
                                           ccf_list=nonsubclass_region_names,
                                           celltype_list=th_celltypes['supertype'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [87]:
supertype_set = supertype_df.query('nucleus_precision>0.6 & nucleus_recall>0.15')
supertype_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
15,RE,0670 RE-Xi Nox4 Glut_3,0.630272,0.624857,0.627552
12,LD,0658 TH Prkcd Grin2c Glut_5,0.678133,0.430846,0.526919
10,MD,0663 TH Prkcd Grin2c Glut_10,0.623616,0.330901,0.432376
20,PVT,0644 PVT-PT Ntrk1 Glut_2,0.707555,0.256932,0.376975
21,PVT,0645 PVT-PT Ntrk1 Glut_3,0.660759,0.230973,0.342295
25,PT,0648 PVT-PT Ntrk1 Glut_6,0.634551,0.161727,0.257760


In [88]:
cluster_df = get_nucleus_celltype_metrics(obs_neurons, ccf_label, 'cluster', 
                                           ccf_list=nonsubclass_region_names,
                                           celltype_list=th_celltypes['cluster'])

/tmp/ipykernel_13095/1653600213.py:33: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2*recall*precision/(recall+precision)


In [89]:
cluster_set = cluster_df.query('nucleus_precision>0.6 & nucleus_recall>0.1')
cluster_set.sort_values('nucleus_f1', ascending=False)

,nucleus,celltype,nucleus_precision,nucleus_recall,nucleus_f1
4,VPM,2649 TH Prkcd Grin2c Glut_1,0.625702,0.654472,0.639764
9,LD,2658 TH Prkcd Grin2c Glut_5,0.841952,0.360912,0.505245
18,PT,2677 TH Prkcd Grin2c Glut_10,0.666189,0.393734,0.494944
13,RE,2698 RE-Xi Nox4 Glut_3,0.670507,0.333333,0.445295
5,MD,2678 TH Prkcd Grin2c Glut_10,0.770477,0.313101,0.445260
16,PF,2724 PF Fzd5 Glut_2,0.708556,0.262897,0.383502
8,PO,2663 TH Prkcd Grin2c Glut_6,0.677262,0.242451,0.357074
24,AMd,2676 TH Prkcd Grin2c Glut_9,0.601824,0.249370,0.352627
14,PVT,2621 PVT-PT Ntrk1 Glut_2,0.772840,0.184661,0.298095
10,LP,2664 TH Prkcd Grin2c Glut_6,0.649559,0.181574,0.283812
